In [0]:
print('Hello world')

# Common

In [0]:

from pyspark.sql import functions as F


# Widgets (env + UC settings)


In [0]:


dbutils.widgets.text("env", "dev")
dbutils.widgets.text("uc_catalog", "my_catalog_dev")
dbutils.widgets.text("uc_schema", "dim")
dbutils.widgets.text("table_name", "dim_date")


In [0]:
env = dbutils.widgets.get("env")
uc_catalog = dbutils.widgets.get("uc_catalog")
uc_schema = dbutils.widgets.get("uc_schema")
table_name = dbutils.widgets.get("table_name")

print("env:", env)
print("catalog:", uc_catalog)
print("schema:", uc_schema)
print("table:", table_name)


In [0]:


# Date range
start_date = "2020-01-01"
end_date   = "2030-12-31"

df_date = (
    spark
    .sql(f"""
        SELECT sequence(
            to_date('{start_date}'),
            to_date('{end_date}'),
            interval 1 day
        ) AS date_array
    """)
    .select(F.explode("date_array").alias("date"))
)

df_date = (
    df_date
    .withColumn("date_key", F.date_format("date", "yyyyMMdd").cast("int"))
    .withColumn("year", F.year("date"))
    .withColumn("month", F.month("date"))
    .withColumn("day", F.dayofmonth("date"))
    .withColumn("day_of_week", F.date_format("date", "E"))
    .withColumn("is_weekend", F.col("day_of_week").isin(["Sat", "Sun"]))
)



In [0]:
spark.sql(f"""
    CREATE catalog IF NOT EXISTS {uc_catalog}""")

In [0]:
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {uc_catalog}.{uc_schema}
""")


# Save in UC

In [0]:
full_table_name = f"{uc_catalog}.{uc_schema}.{table_name}"

(
    df_date
    .write
    .mode("overwrite")         
    .format("delta")
    .saveAsTable(full_table_name)
)

print(f"Saved table: {full_table_name}")
